## Functions to dropletize/photonize (sparse) data

This notebook demonstrates how to treat sparse datasets.
We will start by plotting the data after applying a threshold and make a 'pixelhistogram' (the spectrum of the data in each of the pixels). 
Once can see the single photon peak with a large shoulder on the left side due to cases where the photon deposited energy in neighboring pixels.

We then show how the dropletize this data and plot the spectrum of the 'droplets' - in this log-plot we can clearly see single & multiple photons. There are no signs
photons of a different energy. This confirms that we can use one of two photon reconstruction algorithms.

Please note that this detector has _not_ been converted to keV. For all runs since run 19, all detectors, including the epix100 report their energies in keV!

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')
from tqdm import tqdm
import h5py as h5
from pathlib import Path
import tables
import sys
from importlib import reload

import psana as ps
sys.path.append('/sdf/group/lcls/ds/tools/smalldata_tools/latest')
from smalldata_tools.DetObject import DetObject, DetObjectFunc
from smalldata_tools.ana_funcs.roi_rebin import spectrumFunc
from smalldata_tools.ana_funcs.sparsifyFunc import sparsifyFunc#, unsparsifyFunc

from smalldata_tools.SmallDataUtils import setParameter, getUserData, getUserEnvData

from smalldata_tools.utilities_plotting import hv_image_ctl
from smalldata_tools.utilities_plotting import hv_image
from smalldata_tools.utilities import image_from_dxy
from smalldata_tools.utilities import unsparsify

run = 630 
exp = 'xpptut15'

dsname = 'exp={}:run={}'.format(exp,run,exp[:3],exp)

ds = ps.MPIDataSource(dsname)
#ds.detnames()

# Set detObject and ana function

This is where the detector and the analysis function are defined. Here we specify a 'spectrumFunc' which will make the 'pixelhistogram'.

In [ ]:
detname = 'epix_alc1' #epix100 w/ beamstop
photE=166.
det = DetObject(detname, ds.env(), int(run)) #epix100 w/ beamstop

#plot the pixel histogram
func1_dict = {}
func1_dict['bins'] = np.arange(-1.5,photE*2.5,photE/100).tolist()
func1 = spectrumFunc(**func1_dict)

# add function to detector pipeline
det.addFunc(func1)

userDataCfg = det.params_as_dict()

max_evt = 1
ds.break_after(max_evt) # stop iteration after max_evt events (break statements do not work reliably with MPIDataSource).

userDict = {}
for nevt,evt in tqdm(enumerate(ds.events())):        
    det.getData(evt)
    det.processFuncs()
    userDict[det._name]=getUserData(det)

## Plot detector data of the last event

We deploy a low threshold, largely to keep the color scale in a good range.

In [ ]:
det_evt_thres = det.evt.dat.copy()
thresADU = -99.5
det_evt_thres[det.evt.dat<thresADU]=0 
#hv_image_ctl(image_from_dxy(det_evt_thres,userDataCfg['ix'],userDataCfg['iy']))
hv_image(image_from_dxy(det_evt_thres,userDataCfg['ix'],userDataCfg['iy']), \
         imgLow=-10.5, imgHigh=photE*1.5, cmap='rainbow')

## Plot the pixel histogram 

In [ ]:
plot = hv.Points( (0.5*(userDataCfg['spectrum__spectrum_bins'][:-1]+userDataCfg['spectrum__spectrum_bins'][1:]),\
    np.log(userDict[detname]['spectrum_histogram'])), \
          [('ADU','ADU'),('npix','npix')]).options(width=800)
for i in range(3):
    plot *= hv.VLine(photE*(i+1)).options(line_dash='dotted', color='red')
plot                                                   

## Dropletize the data & look at spectrum

We are finding droplets which is a sets of pixels sharing a boundary above threshold. Pixels sharing an edge with these pixels that fall above a optional second, lower threshold are added - this sharpens the spectrum, but can lead to merged photons should the second threshold be too low. 'thresADU' means that only droplets with a summed energy above the given threshold are kept for further analysis. This is to reject cases where fluctuations have led to pixels above the threshold, but below where we expect to find the photons.

In [ ]:
from smalldata_tools.ana_funcs.droplet import dropletFunc

func2_dict = {}
func2_dict['threshold'] = 3.
func2_dict['thresholdLow'] = 1.
func2_dict['thresADU'] = 4.
func2 = dropletFunc(**func2_dict)

sfunc2 = sparsifyFunc(nData=15000) 
func2.addFunc(sfunc2)

# add function to detector pipeline
det.addFunc(func2)

userDataCfg = det.params_as_dict()
userDict = {}
for nevt,evt in tqdm(enumerate(ds.events())):        
    det.getData(evt)
    det.processFuncs()
    userDict[det._name]=getUserData(det)

In [ ]:
dropletHis = np.histogram(userDict[detname]['droplet_sparse_data'], np.arange(.5,photE*4.5,photE/100))

plot = hv.Points( (0.5*(dropletHis[1][:-1]+dropletHis[1][1:]),\
    np.log(dropletHis[0])), \
          [('dropADU','dropADU'),('ndrop','ndrop')]).options(width=800)
for i in range(5):
    plot *= hv.VLine(photE*(i+1)).options(line_dash='dotted', color='red')
    plot *= hv.VLine(photE*i+photE*0.5).options(line_dash='dashed', color='green', line_width=1)
    #plot *= hv.VLine(photE*i+11.7).options(line_dash='dotted', color='green', line_width=1)
plot

## Make an image with the droplets
In the producer file, you would use the unsparsifyFunc and a (full) ROI w. imageFunc if needed. Similarly, you could use the azimuthal averaging code after the unsparsify.

In [ ]:
droplet_sparse={}
droplet_sparse['data'] = userDict[detname]['droplet_sparse_data']
droplet_sparse['row'] = userDict[detname]['droplet_sparse_row']
droplet_sparse['col'] = userDict[detname]['droplet_sparse_col']
droplet_sparse['tile'] = userDict[detname]['droplet_sparse_tile']

det_drop = unsparsify(droplet_sparse, shape=userDataCfg['mask'].shape)
dropImg = image_from_dxy(det_drop.flatten(), userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())

hv_image_ctl(dropImg)

# Photonize the data (2 pixel, sum > threshold, LCLS1 only)

This algorithm which has only been implemented for LCLS1 defines photons as 1 or two neighboring photons where the energy sum is > 'thr_fraction' of the expected photon energy. It works well in cases where the photons are usually contained in 1 or maybe 2 pixels, but can fail to find photons where a significant fraction of energy is captured in a third pixel.

In [ ]:
from smalldata_tools.ana_funcs.photons import photonFunc

func3_dict = {}
func3_dict['ADU_per_photon'] =photE
func3_dict['thr_fraction'] = 0.85
func3 = photonFunc(**func3_dict)

sfunc3 = sparsifyFunc(nData=15000) 
func3.addFunc(sfunc3)

# add function to detector pipeline
det.addFunc(func3)

userDataCfg = det.params_as_dict()
userDict = {}
# small_data = ds.small_data('./pyfai_dev.h5', gather_interval=5)
for nevt,evt in tqdm(enumerate(ds.events())):        
    det.getData(evt)
    det.processFuncs()
    userDict[det._name]=getUserData(det)
    # small_data.event(userDict)
# small_data.close()

photon_sparse={}
photon_sparse['data'] = userDict[detname]['photon_sparse_data']
photon_sparse['row'] = userDict[detname]['photon_sparse_row']
photon_sparse['col'] = userDict[detname]['photon_sparse_col']
photon_sparse['tile'] = userDict[detname]['photon_sparse_tile']

det_phot = unsparsify(photon_sparse, shape=userDataCfg['mask'].shape)
photImg = image_from_dxy(det_phot.flatten(), userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())

hv_image_ctl(photImg)

# Different approach: photonize the droplet output: place photons inside of the droplet
This approach place single photons inside of multiphoton droplets. Also here, we assume that 2 pixels share the photon energy, but the last photon will use all the remaining energy, so no threshold for the energy contained in 2 pixels is necessary.

In [ ]:
from smalldata_tools.ana_funcs.droplet2Photons import droplet2Photons

func4_dict = {}
func4_dict['photpts'] = (np.arange(1000)-1)*photE + photE/2.
func4 = droplet2Photons(**func4_dict)

sfunc4 = sparsifyFunc(nData=100000) 
func4.addFunc(sfunc4)

func2_dict = {}
func2_dict['threshold'] = 3.
func2_dict['thresholdLow'] = 1.
func2_dict['thresADU'] = 4.
func2 = dropletFunc(**func2_dict)
func2.addFunc(func4)

detname_d2p = '%s_d2p'%detname
det_d2p = DetObject(detname, ds.env(), int(run), name=detname_d2p) #epix100 w/ beamstop
# add function to detector pipeline
det_d2p.addFunc(func2)

userDataCfg_d2p = det_d2p.params_as_dict()
userDict_d2p = {}
# small_data = ds.small_data('./pyfai_dev.h5', gather_interval=5)
for nevt,evt in tqdm(enumerate(ds.events())):        
    det_d2p.getData(evt)
    det_d2p.processFuncs()
    userDict_d2p[det._name]=getUserData(det_d2p)
    # small_data.event(userDict)
# small_data.close()

d2p_sparse={}
d2p_sparse['data'] = userDict_d2p[detname]['droplet_droplet2phot_sparse_data']
d2p_sparse['row'] = userDict_d2p[detname]['droplet_droplet2phot_sparse_row']
d2p_sparse['col'] = userDict_d2p[detname]['droplet_droplet2phot_sparse_col']
d2p_sparse['tile'] = userDict_d2p[detname]['droplet_droplet2phot_sparse_tile']

det_phot = unsparsify(d2p_sparse, shape=userDataCfg['mask'].shape)
photImg = image_from_dxy(det_phot.flatten(), userDataCfg['ix'].flatten(), userDataCfg['iy'].flatten())

hv_image_ctl(photImg)